In [1]:
import pandas as pd
import re


In [ ]:


tf_search = pd.read_csv('SearchResultstranscription factor.tsv', sep='\t')



def interpro_str(x):
    return 'INTERPRO' in x

def tf_str(x):
    return 'transcription factor' in x.lower()

def tf_rgx(x):
    return bool(re.search(r'[tT]ranscription.*[f,F]actor', x))

def t_str(x):
    return 'transcription' in x.lower()


tf_search_ipr = tf_search[tf_search['Source Database'].apply(lambda x: 'INTERPRO' in x)]

tf_search_n_str = tf_search_ipr[tf_search_ipr['Name'].apply(lambda x: not tf_str(x))]

tf_search_n_rgx = tf_search_n_str[tf_search_n_str['Name'].apply(lambda x: not tf_rgx(x))]

In [ ]:
from lxml import etree

def read_xml_with_dtd(xml_file, dtd_file=None):
    """
    Read XML file with DTD validation.

    Parameters
    ----------
    xml_file : str
        Path to the XML file.
    dtd_file : str, optional
        Path to the DTD file. If None, uses DTD reference in the XML.

    Returns
    -------
    pandas.DataFrame or None
    """
    try:
        if dtd_file:
            # Load DTD and parse XML
            with open(dtd_file) as f:
                dtd = etree.DTD(f)

            tree = etree.parse(xml_file)

            # Validate against DTD
            if not dtd.validate(tree):
                print("DTD validation errors:")
                print(dtd.error_log.filter_from_errors())
                return None
        else:
            # Use parser that validates using DOCTYPE reference
            parser = etree.XMLParser(dtd_validation=True)
            tree = etree.parse(xml_file, parser)

        # Convert to DataFrame
        return pd.read_xml(xml_file)

    except Exception as e:
        print(f"Error: {e}")
        return None

ipr_xml = read_xml_with_dtd('interpro.xml', dtd_file='interpro.dtd')